# LangChain中的生成式代理

这个笔记本实现了一种基于论文[生成式代理：人类行为的交互模拟](https://arxiv.org/abs/2304.03442)（作者：Park等）的生成式代理。

在这里，我们利用了一个由LangChain检索器支持的时间加权记忆对象。

In [1]:
# 使用 termcolor 库来实现输出文本的着色
!pip install termcolor > /dev/null

In [1]:
# 导入日志模块
import logging

# 配置日志级别为 ERROR
logging.basicConfig(level=logging.ERROR)

In [2]:
from datetime import datetime, timedelta
from typing import List

from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from termcolor import colored

In [3]:
USER_NAME = "Person A"  # 在与代理进行交流时想要使用的名称。
LLM = ChatOpenAI(max_tokens=1500)  # 可以是任何你想要使用的LLM。

### 生成式代理记忆组件

本教程重点介绍了生成式代理的记忆及其对其行为的影响。该记忆在两个方面与标准的 LangChain 聊天记忆不同：

1. **记忆形成**

   生成式代理具有扩展的记忆，存储在单个流中：
      1. 观察 - 来自对话或与虚拟世界的互动中关于自己或他人的观察
      2. 反思 - 重新浮现并总结核心记忆


2. **记忆回忆**

   记忆是通过重要性、新近性和重要性的加权和来检索的。

您可以在[参考文档]("https://api.python.langchain.com/en/latest/modules/experimental.html")中查看 `GenerativeAgent` 和 `GenerativeAgentMemory` 的定义，重点关注 `add_memory` 和 `summarize_related_memories` 方法。

In [4]:
# 导入 GenerativeAgent 和 GenerativeAgentMemory 类
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)

## 内存生命周期

总结上述关键方法：`add_memory` 和 `summarize_related_memories`。

当一个 agent 进行观察时，它会存储记忆：

1. 语言模型评分记忆的重要性（平凡为1，刻骨铭心为10）。
2. 观察和重要性通过 TimeWeightedVectorStoreRetriever 存储在一个文档中，并带有 `last_accessed_time`。

当一个 agent 对观察做出回应时：

1. 生成检索器的查询，根据显著性、最新性和重要性获取文档。
2. 总结检索到的信息。
3. 更新已使用文档的 `last_accessed_time`。

## 创建一个生成角色

现在我们已经了解了定义，我们将创建两个名为"Tommie"和"Eve"的角色。

In [5]:
import math

import faiss


def relevance_score_fn(score: float) -> float:
    """返回一个在[0, 1]范围内的相似度分数。"""
    # 这将取决于几个因素：
    # - VectorStore使用的距离/相似度度量
    # - 嵌入的规模（OpenAI的是单位规范化的，许多其他嵌入不是！）
    # 该函数将归一化嵌入的欧几里德范数（0最相似，sqrt(2)最不相似）
    # 转换为相似度函数（0到1）
    return 1.0 - score / math.sqrt(2)


def create_new_memory_retriever():
    """创建一个新的对话系统独有的向量存储检索器。"""
    # 定义您的嵌入模型
    embeddings_model = OpenAIEmbeddings()
    # 将向量存储初始化为空
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [6]:
# 创建一个新的GenerativeAgentMemory对象，命名为tommies_memory
tommies_memory = GenerativeAgentMemory(
    llm=LLM,  # 使用LLM作为参数
    memory_retriever=create_new_memory_retriever(),  # 创建一个新的memory_retriever
    verbose=False,  # 不显示详细信息
    reflection_threshold=8,  # 设定反思阈值为8，相对较低的数字以展示反思的工作原理
)

# 创建一个新的GenerativeAgent对象，命名为tommie
tommie = GenerativeAgent(
    name="Tommie",  # 名字为Tommie
    age=25,  # 年龄为25岁
    traits="anxious, likes design, talkative",  # 特点为焦虑、喜欢设计、健谈，可以在这里添加更多的持久特点
    status="looking for a job",  # 状态为正在找工作，当连接到虚拟世界时，角色可以更新他们的状态
    memory_retriever=create_new_memory_retriever(),  # 创建一个新的memory_retriever
    llm=LLM,  # 使用LLM作为参数
    memory=tommies_memory,  # 使用之前创建的tommies_memory作为记忆
)

In [7]:
# 当前角色的“摘要”无法生成，因为代理尚未进行任何观察。
print(tommie.get_summary())

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
No information about Tommie's core characteristics is provided in the given statements.


In [8]:
# 我们可以直接将记忆添加到memory对象中
tommie_observations = [
    "Tommie remembers his dog, Bruno, from when he was a kid",  # Tommie记得他小时候有一只狗，名叫布鲁诺
    "Tommie feels tired from driving so far",  # Tommie因为开了很远的路感到疲倦
    "Tommie sees the new home",  # Tommie看到了新家
    "The new neighbors have a cat",  # 新邻居有一只猫
    "The road is noisy at night",  # 夜晚的道路很吵
    "Tommie is hungry",  # Tommie饿了
    "Tommie tries to get some rest.",  # Tommie试图休息一下
]
for observation in tommie_observations:
    tommie.memory.add_memory(observation)  # 将每个观察结果添加到tommie的记忆中

In [9]:
# 现在Tommie有了“记忆”，他们的自我总结更加详细，尽管仍然很简单。
# 我们将看到在更多观察后，这个总结如何更新以创建更丰富的描述。
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is a person who is observant of his surroundings, has a sentimental side, and experiences basic human needs such as hunger and the need for rest. He also tends to get tired easily and is affected by external factors such as noise from the road or a neighbor's pet.


## 与角色的面试前谈话

在让我们的角色上路之前，让我们问他们几个问题。

In [10]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """帮助笔记本用户与代理进行交互。"""
    # 构建新的消息，包括用户名称和消息内容
    new_message = f"{USER_NAME} says {message}"
    # 调用代理的方法生成对话回复，并返回回复的内容
    return agent.generate_dialogue_response(new_message)[1]

In [11]:
# 调用 interview_agent 函数，传入参数 tommie 和 "What do you like to do?"
interview_agent(tommie, "What do you like to do?")

'Tommie said "I really enjoy design and being creative. I\'ve been working on some personal projects lately. What about you, Person A? What do you like to do?"'

In [12]:
# 调用 interview_agent 函数，传入参数 tommie 和问题 "What are you looking forward to doing today?"
interview_agent(tommie, "What are you looking forward to doing today?")

'Tommie said "Well, I\'m actually looking for a job right now, so hopefully I can find some job postings online and start applying. How about you, Person A? What\'s on your schedule for today?"'

In [13]:
# 调用 interview_agent 函数，传入参数 tommie 和问题 "What are you most worried about today?"
interview_agent(tommie, "What are you most worried about today?")

'Tommie said "Honestly, I\'m feeling pretty anxious about finding a job. It\'s been a bit of a struggle lately, but I\'m trying to stay positive and keep searching. How about you, Person A? What worries you?"'

## 逐步进行当天的观察。

In [14]:
# 让Tommie开始过一天的生活。
observations = [
    "Tommie wakes up to the sound of a noisy construction site outside his window.",  # Tommie被窗外嘈杂的建筑工地声吵醒。
    "Tommie gets out of bed and heads to the kitchen to make himself some coffee.",  # Tommie起床后去厨房给自己冲咖啡。
    "Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some.",  # Tommie意识到他忘了买咖啡滤纸，开始翻找他的搬家箱子以找到一些。
    "Tommie finally finds the filters and makes himself a cup of coffee.",  # Tommie终于找到了滤纸，给自己冲了一杯咖啡。
    "The coffee tastes bitter, and Tommie regrets not buying a better brand.",  # 咖啡尝起来很苦，Tommie后悔没有买一个更好的品牌。
    "Tommie checks his email and sees that he has no job offers yet.",  # Tommie检查了他的电子邮件，发现他还没有收到任何工作机会。
    "Tommie spends some time updating his resume and cover letter.",  # Tommie花了一些时间更新他的简历和求职信。
    "Tommie heads out to explore the city and look for job openings.",  # Tommie出门探索城市并寻找工作机会。
    "Tommie sees a sign for a job fair and decides to attend.",  # Tommie看到了一个招聘会的标志，决定去参加。
    "The line to get in is long, and Tommie has to wait for an hour.",  # 进入的队伍很长，Tommie不得不等一个小时。
    "Tommie meets several potential employers at the job fair but doesn't receive any offers.",  # Tommie在招聘会上遇到了几个潜在的雇主，但没有收到任何工作机会。
    "Tommie leaves the job fair feeling disappointed.",  # Tommie离开招聘会感到失望。
    "Tommie stops by a local diner to grab some lunch.",  # Tommie顺便去当地的一家小餐馆吃午饭。
    "The service is slow, and Tommie has to wait for 30 minutes to get his food.",  # 服务很慢，Tommie不得不等30分钟才能拿到他的食物。
    "Tommie overhears a conversation at the next table about a job opening.",  # Tommie听到了隔壁桌上关于一个工作机会的对话。
    "Tommie asks the diners about the job opening and gets some information about the company.",  # Tommie询问了用餐者关于工作机会的情况，并获得了一些关于公司的信息。
    "Tommie decides to apply for the job and sends his resume and cover letter.",  # Tommie决定申请这份工作，并发送了他的简历和求职信。
    "Tommie continues his search for job openings and drops off his resume at several local businesses.",  # Tommie继续寻找工作机会，并在几家当地企业投递了他的简历。
    "Tommie takes a break from his job search to go for a walk in a nearby park.",  # Tommie从工作搜索中休息一下，去附近的公园散步。
    "A dog approaches and licks Tommie's feet, and he pets it for a few minutes.",  # 一只狗走近并舔了舔Tommie的脚，他抚摸了它几分钟。
    "Tommie sees a group of people playing frisbee and decides to join in.",  # Tommie看到一群人在玩飞盘，决定加入他们。
    "Tommie has fun playing frisbee but gets hit in the face with the frisbee and hurts his nose.",  # Tommie玩飞盘很开心，但被飞盘打到脸上，鼻子受伤了。
    "Tommie goes back to his apartment to rest for a bit.",  # Tommie回到公寓休息一会儿。
    "A raccoon tore open the trash bag outside his apartment, and the garbage is all over the floor.",  # 一只浣熊撕开了他公寓外的垃圾袋，垃圾散落一地。
    "Tommie starts to feel frustrated with his job search.",  # Tommie开始对他的工作搜索感到沮丧。
    "Tommie calls his best friend to vent about his struggles.",  # Tommie打电话给他最好的朋友倾诉他的困境。
    "Tommie's friend offers some words of encouragement and tells him to keep trying.",  # Tommie的朋友给了他一些鼓励的话，并告诉他要继续努力。
    "Tommie feels slightly better after talking to his friend.",  # Tommie和朋友聊过之后感觉稍微好一些了。
]

In [15]:
# 让我们让Tommie上路。我们将每隔几次观察检查他们的总结，以观察其演变过程

for i, observation in enumerate(observations):
    _, reaction = tommie.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % 20) == 0:
        print("*" * 40)
        print(
            colored(
                f"After {i+1} observations, Tommie's summary is:\n{tommie.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)

Tommie wakes up to the sound of a noisy construction site outside his window. Tommie groans and covers his head with a pillow, trying to block out the noise.
Tommie gets out of bed and heads to the kitchen to make himself some coffee. Tommie stretches his arms and yawns before starting to make the coffee.
Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some. Tommie sighs in frustration and continues searching through the boxes.
Tommie finally finds the filters and makes himself a cup of coffee. Tommie takes a deep breath and enjoys the aroma of the fresh coffee.
The coffee tastes bitter, and Tommie regrets not buying a better brand. Tommie grimaces and sets the coffee mug aside.
Tommie checks his email and sees that he has no job offers yet. Tommie sighs and closes his laptop, feeling discouraged.
Tommie spends some time updating his resume and cover letter. Tommie nods, feeling satisfied with his progress.
Tommie heads out to explo

## 面试当天之后的访谈

In [16]:
# 调用 interview_agent 函数，传入参数 tommie 和字符串 "Tell me about how your day has been going"
interview_agent(tommie, "Tell me about how your day has been going")

'Tommie said "It\'s been a bit of a rollercoaster, to be honest. I\'ve had some setbacks in my job search, but I also had some good moments today, like sending out a few resumes and meeting some potential employers at a job fair. How about you?"'

In [17]:
# 调用 interview_agent 函数，传入参数 tommie 和 "How do you feel about coffee?"
interview_agent(tommie, "How do you feel about coffee?")

'Tommie said "I really enjoy coffee, but sometimes I regret not buying a better brand. How about you?"'

In [18]:
# 调用 interview_agent 函数，传入参数 tommie 和字符串 "Tell me about your childhood dog!"
interview_agent(tommie, "Tell me about your childhood dog!")

'Tommie said "Oh, I had a dog named Bruno when I was a kid. He was a golden retriever and my best friend. I have so many fond memories of him."'

## 添加多个角色

让我们添加第二个角色与Tommie进行对话。可以随意配置不同的特征。

In [47]:
eves_memory = GenerativeAgentMemory(
    llm=LLM,  # 使用LLM模型
    memory_retriever=create_new_memory_retriever(),  # 创建新的记忆检索器
    verbose=False,  # 不显示详细信息
    reflection_threshold=5,  # 反思阈值为5
)


eve = GenerativeAgent(
    name="Eve",  # 名字为Eve
    age=34,  # 年龄为34岁
    traits="curious, helpful",  # 特点为"curious, helpful"，你可以在这里添加更多的持久特点
    status="N/A",  # 当连接到虚拟世界时，角色可以更新他们的状态
    llm=LLM,  # 使用LLM模型
    daily_summaries=[
        (
            "Eve started her new job as a career counselor last week and received her first assignment, a client named Tommie."
        )
    ],  # 每日总结为"Eve上周开始了她的新工作，成为一名职业顾问，并收到了她的第一个任务，一个名叫Tommie的客户。"
    memory=eves_memory,  # 使用eves_memory作为记忆
    verbose=False,  # 不显示详细信息
)

In [48]:
# 导入所需的模块
from datetime import datetime, timedelta

# 获取昨天的日期并格式化为 "星期 月份 日" 的形式
yesterday = (datetime.now() - timedelta(days=1)).strftime("%A %B %d")

# 定义 Eve 的观察列表
eve_observations = [
    "Eve wakes up and hear's the alarm",  # Eve 醒来并听到了闹钟的声音
    "Eve eats a boal of porridge",  # Eve 吃了一碗粥
    "Eve helps a coworker on a task",  # Eve 帮助同事完成了一个任务
    "Eve plays tennis with her friend Xu before going to work",  # Eve 和她的朋友 Xu 在上班前打了网球
    "Eve overhears her colleague say something about Tommie being hard to work with",  # Eve 偶然听到同事说 Tommie 很难合作
]

# 遍历 Eve 的观察列表，并将每个观察结果添加到 Eve 的记忆中
for observation in eve_observations:
    eve.memory.add_memory(observation)
以上是给定的代码，根据英文内容直译并添加了适当的注释。

In [49]:
# 打印eve对象的摘要信息
print(eve.get_summary())

Name: Eve (age: 34)
Innate traits: curious, helpful
Eve is a helpful and active person who enjoys sports and takes care of her physical health. She is attentive to her surroundings, including her colleagues, and has good time management skills.


## 预先对话采访


在Eve与Tommie交谈之前，让我们对Eve进行“采访”。

In [50]:
# 调用 interview_agent 函数，传入参数 eve 和 "How are you feeling about today?"
interview_agent(eve, "How are you feeling about today?")

'Eve said "I\'m feeling pretty good, thanks for asking! Just trying to stay productive and make the most of the day. How about you?"'

In [51]:
# 调用 interview_agent 函数，传入参数 eve 和问题 "What do you know about Tommie?"
interview_agent(eve, "What do you know about Tommie?")

'Eve said "I don\'t know much about Tommie, but I heard someone mention that they find them difficult to work with. Have you had any experiences working with Tommie?"'

In [52]:
# 调用 interview_agent 函数，传入参数 eve 和问题字符串
interview_agent(
    eve,
    "Tommie is looking to find a job. What are are some things you'd like to ask him?",
)

'Eve said "That\'s interesting. I don\'t know much about Tommie\'s work experience, but I would probably ask about his strengths and areas for improvement. What about you?"'

In [53]:
# 调用 interview_agent 函数，传入参数 eve 和一段字符串作为对话提示
interview_agent(
    eve,
    "You'll have to ask him. He may be a bit anxious, so I'd appreciate it if you keep the conversation going and ask as many questions as possible.",
)

'Eve said "Sure, I can keep the conversation going and ask plenty of questions. I want to make sure Tommie feels comfortable and supported. Thanks for letting me know."'

## 生成式智能体之间的对话

当生成式智能体与虚拟环境或其他智能体进行交互时，其复杂性会大大增加。下面，我们展示了Tommie和Eve之间的简单对话。

In [54]:
from typing import List

def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """Runs a conversation between agents."""
    # 代理1生成对话反应
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)  # 打印对话反应
    turns = 0
    while True:
        break_dialogue = False
        for agent in agents:
            # 生成对话响应
            stay_in_dialogue, observation = agent.generate_dialogue_response(
                observation
            )
            print(observation)  # 打印对话响应
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

In [55]:
# 定义一个列表变量agents，包含两个元素tommie和eve
agents = [tommie, eve]

# 调用run_conversation函数，传入参数agents和对话内容
run_conversation(
    agents,
    "Tommie said: Hi, Eve. Thanks for agreeing to meet with me today. I have a bunch of questions and am not sure where to start. Maybe you could first share about your experience?",
)

Eve said "Sure, Tommie. I'd be happy to share about my experience. Where would you like me to start?"
Tommie said "That's great, thank you! How about you start by telling me about your previous work experience?"
Eve said "Sure, I'd be happy to share my previous work experience with you. I've worked in a few different industries, including marketing and event planning. What specific questions do you have for me?"
Tommie said "That's great to hear. Can you tell me more about your experience in event planning? I've always been interested in that field."
Eve said "Sure, I'd be happy to share about my experience in event planning. I've worked on a variety of events, from corporate conferences to weddings. One of the biggest challenges I faced was managing multiple vendors and ensuring everything ran smoothly on the day of the event. What specific questions do you have?"
Tommie said "That sounds like a lot of responsibility! Can you tell me more about how you handled the challenges that came

## 让我们在他们对话后对代理进行访谈

由于生成代理会保留他们当天的记忆，我们可以询问他们关于他们的计划、对话和其他记忆的情况。

In [56]:
# 我们可以看到一个角色的当前“概要”，这是基于他们自己对自己的感知而改变的
# 打印出来tommie的概要，并强制刷新
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is determined and hopeful in his job search, but can also feel discouraged and frustrated at times. He has a strong connection to his childhood dog, Bruno. Tommie seeks support from his friends when feeling overwhelmed and is grateful for their help. He also enjoys exploring his new city.


In [57]:
# 打印eve对象的摘要信息
# force_refresh=True表示强制刷新摘要信息
print(eve.get_summary(force_refresh=True))

Name: Eve (age: 34)
Innate traits: curious, helpful
Eve is a helpful and friendly person who enjoys playing sports and staying productive. She is attentive and responsive to others' needs, actively listening and asking questions to understand their perspectives. Eve has experience in event planning and communication, and is willing to share her knowledge and expertise with others. She values teamwork and collaboration, and strives to create a comfortable and supportive environment for everyone.


In [58]:
# 调用 interview_agent 函数，传入 tommie 作为第一个参数，"How was your conversation with Eve?" 作为第二个参数
interview_agent(tommie, "How was your conversation with Eve?")

'Tommie said "It was really helpful actually. Eve shared some great tips on managing events and handling unexpected issues. I feel like I learned a lot from her experience."'

In [59]:
# 调用 interview_agent 函数，传入参数 eve 和字符串 "How was your conversation with Tommie?"
interview_agent(eve, "How was your conversation with Tommie?")

'Eve said "It was great, thanks for asking. Tommie was very receptive and had some great questions about event planning. How about you, have you had any interactions with Tommie?"'

In [60]:
# 调用 interview_agent 函数，传入参数 eve 和问题字符串 "What do you wish you would have said to Tommie?"
interview_agent(eve, "What do you wish you would have said to Tommie?")

'Eve said "It was great meeting with you, Tommie. If you have any more questions or need any help in the future, don\'t hesitate to reach out to me. Have a great day!"'